In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



## Load Data 

In [ ]:


df=pd.read_csv('/kaggle/input/divvy-2019-cyclist-dataset-12month/all_merger.csv')

In [ ]:
print(df.info())
print(len(df))

# convert start and end date in datetime 
# and create two new diffenrent column

In [ ]:
df['start_time1']=pd.to_datetime(df['start_time'],dayfirst=True)
df['end_time1']=pd.to_datetime(df['end_time'],dayfirst=True)

# Get total ride length in seconds

In [ ]:
df['ride_length']=(df['end_time1']-df['start_time1'] ) 
df['ride_length_s']=df.ride_length/np.timedelta64(1,'s')

In [ ]:
df.head()
    

# Random check of trip_id data

In [ ]:
df.loc[df['trip_id']==22219262]

In [ ]:
df_filter=df.loc[df['start_time1']>df['end_time1']]

In [ ]:
len(df_filter)

In [ ]:
df=df.loc[df['start_time1']<df['end_time1']]


# Get Month from start date

In [ ]:
df['month']=df['start_time1'].dt.month_name()

# Get weekday name from start_date column

In [ ]:
df['day']=pd.to_datetime(df['start_time']).dt.day_name()

In [ ]:
df_user_details=df.groupby(['day','usertype']).size()
df_user_details

# Analysis the uses by weekdays of  usertype 

In [ ]:
sns.set_theme(style="whitegrid")
fig_dims=(15,15)
fig, ax = plt.subplots(figsize=fig_dims)
g=sns.countplot(x=df["day"], hue=df['usertype'],data=df,palette="PRGn_r",ax=ax)


for bar in g.patches:
    
  # Using Matplotlib's annotate function and
  # passing the coordinates where the annotation shall be done
  # x-coordinate: bar.get_x() + bar.get_width() / 2
  # y-coordinate: bar.get_height()
  # free space to be left to make graph pleasing: (0, 8)
  # ha and va stand for the horizontal and vertical alignment
    g.annotate(format(bar.get_height(), '.2f'), 
                   (bar.get_x() + bar.get_width() / 2, 
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points',color='blue')

**Get Mean ,Max and Min ride_length from total_data**

In [ ]:
mean_ride_length=df['ride_length_s'].mean()
max_ride_length=df['ride_length_s'].max()
min_ride_length=df['ride_length_s'].min()


In [ ]:
print(mean_ride_length,'\n',max_ride_length,'\n',min_ride_length)

In [ ]:
# Details of max_ride length from data
df.loc[df['ride_length_s']==max_ride_length]

# create two sub_data set by usertype 

## 1) Dataframe of Subscriber user
## 2) Dataframe of Customer user


In [ ]:
df_customer=df.loc[df['usertype']=='Customer']
df_sub=df.loc[df['usertype']=='Subscriber']

In [ ]:
sns.set_theme(style="whitegrid")
fig_dims=(12,12)
fig, ax = plt.subplots(figsize=fig_dims)
g=sns.countplot(x=df["day"], hue=df['usertype'],data=df,palette="Set2",ax=ax)


for bar in g.patches:
    
  # Using Matplotlib's annotate function and
  # passing the coordinates where the annotation shall be done
  # x-coordinate: bar.get_x() + bar.get_width() / 2
  # y-coordinate: bar.get_height()
  # free space to be left to make graph pleasing: (0, 8)
  # ha and va stand for the horizontal and vertical alignment
    g.annotate(format(bar.get_height(), '.2f'), 
                   (bar.get_x() + bar.get_width() / 2, 
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points',color='blue')

**From Above Graph Subscriber user don't use service in weekend , might be they use weekedays for daily activities
 but in cutomers user type  opposite reflection they use in weekend compare to other days.******



In [ ]:
df['ride_leng_e']=df['end_time1']-df['start_time1']

In [ ]:
mean_ride_length=df['ride_length'].mean()
max_ride_length=df['ride_length'].max()
min_ride_length=df['ride_length'].min()

In [ ]:
print(mean_ride_length,'\n',max_ride_length,'\n',min_ride_length)

In [ ]:

df.loc[df['ride_length'] == max_ride_length]

**Get mean,max and min ride_length from customer dataframe**


In [ ]:
df_cus_mean=df_customer.groupby(['day'])['ride_length_s'].mean().reset_index()

df_cus_max=df_customer.groupby(['day'])['ride_length_s'].max().reset_index()

df_cus_min=df_customer.groupby(['day'])['ride_length_s'].min().reset_index()

**Get mean,max and min ride_length from subscriber dataframe**

In [ ]:
df_sub_mean=df_sub.groupby(['day'])['ride_length_s'].mean().reset_index()


df_sub_max=df_sub.groupby(['day'])['ride_length_s'].max().reset_index()

df_sub_min=df_sub.groupby(['day'])['ride_length_s'].min().reset_index()


## merge both usertype data by mean,max and min ride_length value and plot 

In [ ]:
mean_ride_user = pd.merge(left=df_cus_mean, right=df_sub_mean, left_on='day', right_on='day')

In [ ]:

mean_ride_user.plot(kind='bar',figsize=(10,10)).set_xticklabels(mean_ride_user['day'])

plt.legend(['Sub','Cus'])
plt.title('Sub Vs Cus Mean uses')
plt.ylabel("Time (Seconds)")

In [ ]:
max_ride_user = pd.merge(left=df_cus_max, right=df_sub_max, left_on='day', right_on='day')

In [ ]:
max_ride_user.plot(kind='bar',figsize=(10,10)).set_xticklabels(max_ride_user['day'])
plt.legend(['Sub','Cus'])
plt.title('Sub Vs Cus Max uses')
plt.ylabel("Time (Seconds)")

In [ ]:
min_ride_user = pd.merge(left=df_cus_min, right=df_sub_min, left_on='day', right_on='day')

In [ ]:
min_ride_user.plot(kind='bar',figsize=(10,10)).set_xticklabels(min_ride_user['day'])
plt.legend(['Sub','Cus'])
plt.title('Sub Vs Cus Min uses')
plt.ylabel("Time (Seconds)")

In [ ]:
df_user_details_month=df.groupby(['month','usertype']).size()
df_user_details_month

In [ ]:
sns.set_theme(style="darkgrid")
fig_dims=(20,15)
fig, ax = plt.subplots(figsize=fig_dims)
g=sns.countplot(x=df["month"], hue=df['usertype'],data=df,palette="PuRd",ax=ax)


for bar in g.patches:
    
  # Using Matplotlib's annotate function and
  # passing the coordinates where the annotation shall be done
  # x-coordinate: bar.get_x() + bar.get_width() / 2
  # y-coordinate: bar.get_height()
  # free space to be left to make graph pleasing: (0, 8)
  # ha and va stand for the horizontal and vertical alignment
    g.annotate(format(bar.get_height(), '.2f'), 
                   (bar.get_x() + bar.get_width() / 2, 
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points',color='black')

##  In Q3 of  month of July ,August and September both  Sub and Customer usertype are  highest number 

---



---



---

****